# Week 8 Notebook: Model training, hyperparameter tuning, and model evaluation
The goal of this week's assignment is to use a third modeling method with 3 different hyperparameter settings of the method. 

### Import packages

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
import warnings
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, root_mean_squared_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
#from xgboost import XGBRegressor

### Read data as dataframe

In [5]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

data_folder = os.path.join(parent_dir,"data")
raw_data_folder = os.path.join(data_folder,"raw")
interim_data_folder = os.path.join(data_folder,"interim")
processed_data_folder = os.path.join(data_folder, "processed")

In [6]:
# X data path
X_train_scaled_path = os.path.join(processed_data_folder, 'X_train_scaled.parquet')
X_val_scaled_path = os.path.join(processed_data_folder, 'X_val_scaled.parquet')
X_test_scaled_path = os.path.join(processed_data_folder, 'X_test_scaled.parquet')

train_pca_path = os.path.join(processed_data_folder, 'X_train_pca.parquet')
val_pca_path = os.path.join(processed_data_folder, 'X_val_pca.parquet')
test_pca_path = os.path.join(processed_data_folder, 'X_test_pca.parquet')
# Y data path
y_train_path = os.path.join(processed_data_folder, 'y_train.parquet')
y_val_path = os.path.join(processed_data_folder, 'y_val.parquet')
y_test_path = os.path.join(processed_data_folder, 'y_test.parquet')

In [7]:
# Reading the parquet files as dataframes
X_train_scaled = pd.read_parquet(X_train_scaled_path)
X_val_scaled = pd.read_parquet(X_val_scaled_path)
X_test_scaled = pd.read_parquet(X_test_scaled_path)

y_train = pd.read_parquet(y_train_path)
y_val = pd.read_parquet(y_val_path)
y_test = pd.read_parquet(y_test_path)

X_train_pca = pd.read_parquet(train_pca_path)
X_val_pca = pd.read_parquet(val_pca_path)
X_test_pca = pd.read_parquet(test_pca_path)

### Modeling

### Random Forest

In [5]:
# Define parameter combinations to try
param_combinations = [
    {'n_estimators': 50, 'max_depth': None, 'max_features': None},
    {'n_estimators': 100, 'max_depth': 10, 'max_features': 'sqrt'},
    {'n_estimators': 150, 'max_depth': 20, 'max_features': 'log2'}
]

# Initialize variables to store best model and score
best_model = None
best_params = None
lowest_mse = float('inf')

# Loop through each parameter combination
for params in param_combinations:
    # Initialize the model with the current parameters
    rf_model = RandomForestRegressor(
        n_estimators=params['n_estimators'],
        max_depth=params['max_depth'],
        max_features=params['max_features'],
        random_state=42
    )
    
    # Train the model
    rf_model.fit(X_train_scaled, y_train)
    
    # Predict on validation data
    y_val_pred = rf_model.predict(X_val_scaled)
    
    # Calculate Mean Squared Error
    mse = mean_squared_error(y_val, y_val_pred)
    
    # Check if this is the best model so far
    if mse < lowest_mse:
        lowest_mse = mse
        best_model = rf_model
        best_params = params

# Print the best hyperparameters and score
print("Best Parameters:", best_params)
print("Lowest MSE on validation set:", lowest_mse)


c:\Users\holly\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\holly\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\holly\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Best Parameters: {'n_estimators': 50, 'max_depth': None, 'max_features': None}
Lowest MSE on validation set: 3.40445860460778


In [11]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Define parameter combinations to try
param_combinations = [
    {'n_estimators': 50, 'max_depth': None, 'max_features': None},
    {'n_estimators': 100, 'max_depth': 10, 'max_features': 'sqrt'},
    {'n_estimators': 150, 'max_depth': 20, 'max_features': 'log2'}
]

# Initialize variables to store the best model and score
best_model = None
best_params = None
lowest_mse = float('inf')

# Loop through each parameter combination
for params in param_combinations:
    # Initialize the model with the current parameters
    rf_model = RandomForestRegressor(
        n_estimators=params['n_estimators'],
        max_depth=params['max_depth'],
        max_features=params['max_features'],
        random_state=42
    )
    
    # Train the model
    rf_model.fit(X_train_scaled, y_train.values.ravel())
    
    # Predict on training data
    y_train_pred = rf_model.predict(X_train_scaled)
    
    # Predict on validation data
    y_val_pred = rf_model.predict(X_val_scaled)
    
    # Calculate metrics for training data
    mse_train = mean_squared_error(y_train, y_train_pred)
    rmse_train = np.sqrt(mse_train)
    r2_train = r2_score(y_train, y_train_pred)
    
    # Calculate metrics for validation data
    mse_val = mean_squared_error(y_val, y_val_pred)
    rmse_val = np.sqrt(mse_val)
    r2_val = r2_score(y_val, y_val_pred)
    
    # Check if this is the best model so far (based on validation MSE)
    if mse_val < lowest_mse:
        lowest_mse = mse_val
        best_model = rf_model
        best_params = params

    # Print metrics for the current model
    print(f"Parameters: {params}")
    print(f"Training MSE: {mse_train:.4f}, RMSE: {rmse_train:.4f}, R²: {r2_train:.4f}")
    print(f"Validation MSE: {mse_val:.4f}, RMSE: {rmse_val:.4f}, R²: {r2_val:.4f}")
    print("-" * 50)

# Print the best hyperparameters and score
print("Best Parameters:", best_params)
print("Lowest MSE on validation set:", lowest_mse)

Parameters: {'n_estimators': 50, 'max_depth': None, 'max_features': None}
Training MSE: 0.5153, RMSE: 0.7178, R²: 0.9999
Validation MSE: 3.2682, RMSE: 1.8078, R²: 0.9996
--------------------------------------------------
Parameters: {'n_estimators': 100, 'max_depth': 10, 'max_features': 'sqrt'}
Training MSE: 2115.6830, RMSE: 45.9966, R²: 0.7581
Validation MSE: 2059.3197, RMSE: 45.3797, R²: 0.7473
--------------------------------------------------
Parameters: {'n_estimators': 150, 'max_depth': 20, 'max_features': 'log2'}
Training MSE: 616.2523, RMSE: 24.8244, R²: 0.9295
Validation MSE: 1054.9498, RMSE: 32.4800, R²: 0.8706
--------------------------------------------------
Best Parameters: {'n_estimators': 50, 'max_depth': None, 'max_features': None}
Lowest MSE on validation set: 3.2682202267105542


### XGBoost

In [6]:
# Define parameter combinations to try
param_combinations = [
    {'n_estimators': 50, 'max_depth': 3, 'learning_rate': 0.1},
    {'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.05},
    {'n_estimators': 150, 'max_depth': 7, 'learning_rate': 0.01}
]

# Initialize variables to store best model and score
best_model = None
best_params = None
lowest_mse = float('inf')

# Loop through each parameter combination
for params in param_combinations:
    # Initialize the model with the current parameters
    xgb_model = XGBRegressor(
        n_estimators=params['n_estimators'],
        max_depth=params['max_depth'],
        learning_rate=params['learning_rate'],
        random_state=42,
        verbosity=0  # Suppresses warning messages for cleaner output
    )
    
    # Train the model
    xgb_model.fit(X_train_scaled, y_train)
    
    # Predict on validation data
    y_val_pred = xgb_model.predict(X_val_scaled)
    
    # Calculate Mean Squared Error
    mse = mean_squared_error(y_val, y_val_pred)
    
    # Check if this is the best model so far
    if mse < lowest_mse:
        lowest_mse = mse
        best_model = xgb_model
        best_params = params

# Print the best hyperparameters and score
print("Best Parameters for XGBoost:", best_params)
print("Lowest MSE on validation set for XGBoost:", lowest_mse)

Best Parameters for XGBoost: {'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.05}
Lowest MSE on validation set for XGBoost: 58.14836837530017
